In [ ]:
#@markdown **RUN ME**
BOT_TOKEN = "00000:000000"
ADMIN_CHAT_ID = 000000000
STRENGTH = 0.5
ANIMATE = False
WIDTH, HEIGHT = 512, 512
IS_BUSY = False
FRAMES = 60
CADENCE = 1

import subprocess, time, gc, os, sys

start_time = time.time()
sub_p_res = subprocess.run(
    [
        "nvidia-smi",
        "--query-gpu=name,memory.total,memory.free",
        "--format=csv,noheader",
    ],
    stdout=subprocess.PIPE,
).stdout.decode("utf-8")


packages = [
      "triton xformers",
      "einops==0.4.1 pytorch-lightning==1.7.7 torchdiffeq==0.2.3 torchsde==0.2.5",
      "ftfy timm transformers open-clip-torch omegaconf torchmetrics==0.11.4",
      "safetensors kornia accelerate jsonmerge matplotlib resize-right",
      "scikit-learn numpngw pydantic",
      "telebot",
  ]
for package in packages:
      print(f"..installing {package}")
      subprocess.check_call(
          [sys.executable, "-m", "pip", "install"] + package.split()
      )
if not os.path.exists("deforum-stable-diffusion"):
      subprocess.check_call(
          [
              "git",
              "clone",
              "-b",
              "0.7.1",
              "https://github.com/deforum-art/deforum-stable-diffusion.git",
          ]
      )

with open("deforum-stable-diffusion/src/k_diffusion/__init__.py", "w") as f:
      f.write("")
      sys.path.extend(
      [
          "deforum-stable-diffusion/",
          "deforum-stable-diffusion/src",
      ]
  )



import clip
from helpers.save_images import get_output_folder
from helpers.settings import load_args
from helpers.render import (
    render_animation,
    render_input_video,
    render_image_batch,
    render_interpolation,
)
from helpers.model_load import (
    make_linear_decode,
    load_model,
    get_model_output_paths,
)
from helpers.aesthetics import load_aesthetics_model
from helpers.prompts import Prompts
import torch
import random
from helpers.ffmpeg_helpers import (
    get_extension_maxframes,
    get_auto_outdir_timestring,
    get_ffmpeg_path,
    make_mp4_ffmpeg,
    make_gif_ffmpeg,
    patrol_cycle,
)
from IPython import display
from types import SimpleNamespace
def PathSetup():
    models_path = "models"
    configs_path = "configs"
    output_path = "outputs"
    mount_google_drive = False
    models_path_gdrive = "/content/AI/models"
    output_path_gdrive = "/content/AI/StableDiffusion"
    return locals()
root = SimpleNamespace(**PathSetup())
root.models_path, root.output_path = get_model_output_paths(root)

def ModelSetup():
    map_location = "cuda"  # ["cpu", "cuda"]
    model_config = "v1-inference.yaml"  # ["custom","v2-inference.yaml","v2-inference-v.yaml","v1-inference.yaml"]
    model_checkpoint = "Protogen_V2.2.ckpt"  # ["custom","v2-1_768-ema-pruned.ckpt","v2-1_512-ema-pruned.ckpt","768-v-ema.ckpt","512-base-ema.ckpt","Protogen_V2.2.ckpt","v1-5-pruned.ckpt","v1-5-pruned-emaonly.ckpt","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","wd-v1-3-float16.ckpt"]
    #dreamshaper-v6
    model_checkpoint = "dreamshaper-xl-v2-turbo.ckpt"
    custom_config_path = ""
    custom_checkpoint_path = ""
    return locals()
root.__dict__.update(ModelSetup())
root.model, root.device = load_model(
    root, load_on_run_all=True, check_sha256=True, map_location=root.map_location
)


import telebot
from pathlib import Path
import time
from PIL import Image
from telebot import types
from google.colab import runtime
import datetime

bot = telebot.TeleBot(BOT_TOKEN)

def DeforumAnimArgs():
    animation_mode = (
        "None"  # @ ['None', '2D', '3D', 'Video Input', 'Interpolation'] {type:'string'}
    )
    max_frames = 60  # @ {type:"number"}
    border = "replicate"
    diffusion_cadence = "2"  # @ {type:"string"}
    angle = "0:(0)"
    zoom = "0:(1.04)"
    translation_x = "0:(0)"  # @  #to the right move
    # translation_x = "0:(10*sin(2*3.14*t/10))"
    translation_y = "0:(0)"  # @
    translation_z = "0:(-1)"  # @ #to the forward move
    rotation_3d_x = "0:(0)"
    rotation_3d_y = "0:(0)"
    rotation_3d_z = "0:(0)"
    flip_2d_perspective = False
    perspective_flip_theta = "0:(0)"
    perspective_flip_phi = "0:(t%15)"
    perspective_flip_gamma = "0:(0)"
    perspective_flip_fv = "0:(53)"
    noise_schedule = "0: (0.02)"
    strength_schedule = "0: (0.65)"
    contrast_schedule = "0: (1.0)"
    hybrid_comp_alpha_schedule = "0:(1)"
    hybrid_comp_mask_blend_alpha_schedule = "0:(0.5)"
    hybrid_comp_mask_contrast_schedule = "0:(1)"
    hybrid_comp_mask_auto_contrast_cutoff_high_schedule = "0:(100)"
    hybrid_comp_mask_auto_contrast_cutoff_low_schedule = "0:(0)"

    enable_schedule_samplers = False
    sampler_schedule = "0:('euler'),10:('dpm2'),20:('dpm2_ancestral'),30:('heun'),40:('euler'),50:('euler_ancestral'),60:('dpm_fast'),70:('dpm_adaptive'),80:('dpmpp_2s_a'),90:('dpmpp_2m')"

    kernel_schedule = "0: (5)"
    sigma_schedule = "0: (1.0)"
    amount_schedule = "0: (0.2)"
    threshold_schedule = "0: (0.0)"

    color_coherence = "Match Frame 0 LAB"
    color_coherence_video_every_N_frames = 1
    color_force_grayscale = False

    use_depth_warping = True
    midas_weight = 0.3
    near_plane = 200
    far_plane = 10000
    fov = 40
    padding_mode = "border"
    sampling_mode = "bicubic"
    save_depth_maps = False

    video_init_path = "/content/IMG_4024.MOV"  # @ {type:"string"}
    extract_nth_frame = 2  # @ {type:"number"}
    overwrite_extracted_frames = True
    use_mask_video = False
    video_mask_path = ""

    hybrid_generate_inputframes = False
    hybrid_use_first_frame_as_init_image = True
    hybrid_motion = "None"
    hybrid_motion_use_prev_img = False
    hybrid_flow_method = "DIS Medium"
    hybrid_composite = False
    hybrid_comp_mask_type = "None"
    hybrid_comp_mask_inverse = False
    hybrid_comp_mask_equalize = "None"
    hybrid_comp_mask_auto_contrast = False
    hybrid_comp_save_extra_frames = False
    hybrid_use_video_as_mse_image = False

    interpolate_key_frames = False
    interpolate_x_frames = 20

    resume_from_timestring = False
    resume_timestring = ""

    return locals()


def DeforumArgs(global_batch_name):
    strength = 0.5  # @ {type:"number"}
    W = 512  # @
    H = 512  # @
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64
    bit_depth_output = 8

    seed = -1
    sampler = "euler_ancestral"
    steps = 40
    scale = 7
    ddim_eta = 0.0
    dynamic_threshold = None
    static_threshold = None

    save_samples = True
    save_settings = True
    display_samples = True
    save_sample_per_step = False
    show_sample_per_step = False

    n_batch = 1
    n_samples = 1
    batch_name = global_batch_name
    filename_format = "{timestring}_{index}_{prompt}.png"

    seed_behavior = "random"  # @ ["iter","fixed","random","ladder","alternate"]
    seed_iter_N = 1
    make_grid = False
    grid_rows = 3
    outdir = get_output_folder(root.output_path, batch_name)

    use_init = False  # @ {type:"boolean"}
    strength_0_no_init = True
    init_image = "/content/Screenshot 2023-06-22 224919.png"  # @ {type:"string"}
    add_init_noise = False
    init_noise = 0.01
    use_mask = False
    use_alpha_as_mask = False
    mask_file = ""
    invert_mask = False
    mask_brightness_adjust = 1.0
    mask_contrast_adjust = 1.0
    overlay_mask = True
    mask_overlay_blur = 5

    mean_scale = 0
    var_scale = 0
    exposure_scale = 0
    exposure_target = 0.5

    colormatch_scale = 0
    colormatch_image = ""
    colormatch_n_colors = 4
    ignore_sat_weight = 0

    clip_name = "ViT-L/14"
    clip_scale = 0
    aesthetics_scale = 0
    cutn = 1
    cut_pow = 0.0001

    init_mse_scale = 0
    init_mse_image = ""
    blue_scale = 0

    gradient_wrt = "x0_pred"
    gradient_add_to = "both"
    decode_method = "linear"
    grad_threshold_type = "dynamic"
    clamp_grad_threshold = 0.2
    clamp_start = 0.2
    clamp_stop = 0.01
    grad_inject_timing = list(range(1, 10))

    cond_uncond_sync = True
    precision = "autocast"
    C = 4
    f = 8

    cond_prompt = ""
    cond_prompts = ""
    uncond_prompt = ""
    uncond_prompts = ""
    timestring = ""
    init_latent = None
    init_sample = None
    init_sample_raw = None
    mask_sample = None
    init_c = None
    seed_internal = 0

    return locals()


def ffmpegArgs(global_batch_name, args):
    yyyymm = datetime.date.today().strftime("%Y-%m")
    ffmpeg_mode = "auto"
    ffmpeg_outdir = ""
    ffmpeg_timestring = ""
    ffmpeg_image_path = f"/content/outputs/{yyyymm}/{global_batch_name}"
    ffmpeg_mp4_path = ""
    ffmpeg_gif_path = ""
    ffmpeg_extension = "png"
    ffmpeg_maxframes = 500
    ffmpeg_fps = 12  # @
    ffmpeg_outdir, ffmpeg_timestring = get_auto_outdir_timestring(args, ffmpeg_mode)
    ffmpeg_extension, ffmpeg_maxframes = get_extension_maxframes(
        args, ffmpeg_outdir, ffmpeg_timestring
    )
    ffmpeg_image_path, ffmpeg_mp4_path, ffmpeg_gif_path = get_ffmpeg_path(
        ffmpeg_outdir, ffmpeg_timestring, ffmpeg_extension
    )
    return locals()


# ALL HAPPENS HERE
def DO_MAGIC(msgtext, message_chat_id, file=None):
    global IS_BUSY, FRAMES, STRENGTH, WIDTH, HEIGHT, CADENCE

    if IS_BUSY:
        bot.send_message(message_chat_id, "BUSY =( ")
        return

    print("-----------------------DOING MAGIC!-------------------------")
    IS_BUSY = True
    neg_prompts = {"disfigured, ugly, distorted"}
    global_batch_name = f"{message_chat_id}/sd_output_{round(time.time() * 1000)}"

    args_dict = DeforumArgs(global_batch_name)
    anim_args_dict = DeforumAnimArgs()

    args = SimpleNamespace(**args_dict)
    anim_args = SimpleNamespace(**anim_args_dict)

    args.timestring = time.strftime("%Y%m%d%H%M%S")
    args.strength = max(0.0, min(1.0, STRENGTH))
    args.use_init = False
    args.W = WIDTH
    args.H = HEIGHT
    prompt = f"{msgtext}, high resolution"



    if ANIMATE:
        anim_args.max_frames = FRAMES
        anim_args.animation_mode = "3D"
        anim_args.diffusion_cadence = f"{CADENCE}"
    else:
        anim_args.animation_mode = "None"
        anim_args.max_frames = 1

    if file:
        args.init_image = file
        anim_args.video_init_path = file
        args.use_init = True

        if file.endswith("mp4"):
            anim_args.animation_mode = "Video Input"
            args.use_init = False

    ww, hh = map(lambda x: x - x % 64, (args.W, args.H))
    args.W = ww
    args.H = hh

    # Load clip model if using clip guidance
    if (args.clip_scale > 0) or (args.aesthetics_scale > 0):
        root.clip_model = (
            clip.load(args.clip_name, jit=False)[0]
            .eval()
            .requires_grad_(False)
            .to(root.device)
        )
        if args.aesthetics_scale > 0:
            root.aesthetics_model = load_aesthetics_model(args, root)

    if args.seed == -1:
        args.seed = random.randint(0, 2**32 - 1)
    if not args.use_init:
        args.init_image = None
    if args.sampler == "plms" and (args.use_init or anim_args.animation_mode != "None"):
        args.sampler = "klms"
    if args.sampler != "ddim":
        args.ddim_eta = 0

    # clean up unused memory
    gc.collect()
    torch.cuda.empty_cache()

    # get prompts
    cond, uncond = Prompts(prompt=prompt, neg_prompt=neg_prompts).as_dict()

    # dispatch to appropriate renderer
    bot.send_message(message_chat_id, "start render")
    if anim_args.animation_mode == "2D" or anim_args.animation_mode == "3D":
        render_animation(root, anim_args, args, cond, uncond)
    elif anim_args.animation_mode == "Video Input":
        render_input_video(root, anim_args, args, cond, uncond)
    elif anim_args.animation_mode == "Interpolation":
        render_interpolation(root, anim_args, args, cond, uncond)
    else:
        render_image_batch(root, args, cond, uncond)
        bot.send_message(message_chat_id, "end render")

    yyyymm = datetime.date.today().strftime("%Y-%m")

    folder_path = f"/content/outputs/{yyyymm}/{global_batch_name}"

    if anim_args.animation_mode != "None":
        bot.send_message(message_chat_id, "start ffmpeg")
        ffmpeg_args_dict = ffmpegArgs(global_batch_name, args)
        ffmpeg_args = SimpleNamespace(**ffmpeg_args_dict)
        make_mp4_ffmpeg(ffmpeg_args, display_ffmpeg=True, debug=False)
        bot.send_message(message_chat_id, "end ffmpeg")

        image_filenames = [
            filename
            for filename in os.listdir(folder_path)
            if any(extension in filename for extension in [".mp4"])
        ]
    else:
        image_filenames = [
            filename
            for filename in os.listdir(folder_path)
            if any(extension in filename for extension in [".jpeg", ".jpg", ".png"])
        ]

    # Loop through image filenames and send each as a photo
    for filename in image_filenames:
        # Open image and convert to bytes object
        with open(os.path.join(folder_path, filename), "rb") as file:
            image_bytes = file.read()
            if any(extension in filename for extension in [".mp4"]):
                bot.send_video(
                    chat_id=message_chat_id, video=image_bytes, caption=prompt
                )
                bot.send_document(
                    chat_id=message_chat_id, document=image_bytes, caption=prompt
                )
            if any(extension in filename for extension in [".jpeg", ".jpg", ".png"]):
                bot.send_photo(
                    chat_id=message_chat_id, photo=image_bytes, caption=prompt
                )
    IS_BUSY = False


# Define handler for file messages
@bot.message_handler(content_types=["video"])
def save_file(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")


    file_id = message.video.file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    Path(f"/content/inputs/{message.chat.id}").mkdir(parents=True, exist_ok=True)
    file_name = f"/content/inputs/{message.chat.id}/{round(time.time() * 1000)}.mp4"

    with open(file_name, "wb") as new_file:
        new_file.write(downloaded_file)

    DO_MAGIC(message.caption, message.chat.id, file_name)
    # bot.reply_to(message, message.caption)


# Define handler for photo messages
@bot.message_handler(content_types=["photo"])
def save_photo(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    # Save the photo with the same file name as received
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    Path(f"/content/inputs/{message.chat.id}").mkdir(parents=True, exist_ok=True)
    file_name = f"/content/inputs/{message.chat.id}/{round(time.time() * 1000)}.jpg"

    with open(file_name, "wb") as new_file:
        new_file.write(downloaded_file)

    DO_MAGIC(message.caption, message.chat.id, file_name)
    # bot.reply_to(message, message.caption)


@bot.message_handler(commands=["cadence"])
def handle_frames(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    global CADENCE
    keyboard = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    keyboard.add("Cadence: 1")
    keyboard.add("Cadence: 2")
    keyboard.add("Cadence: 3")
    keyboard.add("Cadence: 4")
    keyboard.add("Cadence: 5")
    keyboard.add("Cadence: 6")
    keyboard.add("Cadence: 7")
    keyboard.add("Cadence: 8")
    bot.send_message(message.chat.id, "Cadence: ", reply_markup=keyboard)


@bot.message_handler(commands=["frames"])
def handle_frames(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    global FRAMES
    keyboard = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    keyboard.add("Frames: 36")
    keyboard.add("Frames: 60")
    keyboard.add("Frames: 120")
    bot.send_message(message.chat.id, "Frames: ", reply_markup=keyboard)


@bot.message_handler(commands=["strength"])
def handle_strength(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    global STRENGTH
    keyboard = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    keyboard.add("Strength: 0.01")
    keyboard.add("Strength: 0.1")
    keyboard.add("Strength: 0.2")
    keyboard.add("Strength: 0.3")
    keyboard.add("Strength: 0.4")
    keyboard.add("Strength: 0.5")
    keyboard.add("Strength: 0.6")
    keyboard.add("Strength: 0.7")
    keyboard.add("Strength: 0.8")
    keyboard.add("Strength: 0.9")
    keyboard.add("Strength: 0.99")
    bot.send_message(message.chat.id, "Strength: ", reply_markup=keyboard)


@bot.message_handler(commands=["animate"])
def handle_animate(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    keyboard = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    keyboard.add("Animate: True")
    keyboard.add("Animate: False")
    bot.send_message(message.chat.id, "Animate: ", reply_markup=keyboard)


@bot.message_handler(commands=["size"])
def handle_size(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    keyboard = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    size_options = ["Size: 360x540", "Size: 540x360", "Size: 512x512", "Size: 720x1080", "Size: 1080x720"]
    keyboard.add(*[types.KeyboardButton(option) for option in size_options])
    bot.send_message(message.chat.id, "Please select a size:", reply_markup=keyboard)


@bot.message_handler(commands=["start"])
def handle_size(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    bot.reply_to(
        message,
        "Welcome! \\r\\n use commands \\size \\animate \\strength \\frames \\cadence",
    )

@bot.message_handler(commands=["stop"])
def handle_stop(message):
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    bot.reply_to(
        message,
        "stopping runtime",
    )

    runtime.unassign()


# Define handler for text messages
@bot.message_handler(func=lambda message: True)
def save_text(message):
    global WIDTH, HEIGHT, ANIMATE, STRENGTH, FRAMES, CADENCE
    if message.chat.id != ADMIN_CHAT_ID:
        bot.reply_to(message, "Sorry, this bot is only for me 👻")

    if message.text.startswith("Cadence:"):
        command, arg = message.text.split(" ")
        CADENCE = int(arg)
        bot.reply_to(message, f"Cadence set to: {CADENCE}")
    elif message.text.startswith("Frames:"):
        command, arg = message.text.split(" ")
        FRAMES = int(arg)
        bot.reply_to(message, f"Frames set to: {FRAMES}")
    elif message.text.startswith("Strength:"):
        command, arg = message.text.split(" ")
        STRENGTH = float(arg)
        bot.reply_to(message, f"Strength set to: {STRENGTH}")
    elif message.text == "Animate: True":
        ANIMATE = True
        bot.reply_to(message, f"Animate set to: {ANIMATE}")
    elif message.text == "Animate: False":
        ANIMATE = False
        bot.reply_to(message, f"Animate set to:  {ANIMATE}")
    elif message.text.startswith("Size:"):
        command, arg = message.text.split(" ")
        www, hhh = arg.split("x")
        WIDTH = int(www)
        HEIGHT = int(hhh)
        bot.reply_to(message, f"Size selected: {WIDTH}x{HEIGHT}")
    else:
        DO_MAGIC(message.text, message.chat.id)


def skip_updates():
    updates = bot.get_updates()
    if updates:
        last_update_id = updates[-1].update_id
        bot.get_updates(offset=last_update_id + 1)

end_time = time.time()
print(f"..environment set up in {end_time-start_time:.0f} seconds")

if __name__ == "__main__":
    skip_updates()
    bot.send_message(
        ADMIN_CHAT_ID,
        f'Welcome! ..environment set up in {end_time-start_time:.0f} seconds',
    )

    while True:
        try:
            bot.polling(none_stop=True)
        except Exception as e:
            print("Error:", e)
            # Wait for few seconds before retrying
            time.sleep(5)
